In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 19 21:46:31 2019

@author: Piyushkumar
"""

import tensorflow as tf 
import numpy as np 
import cv2
import pickle
import random
import models
import copy
import time
import math

In [2]:
from PIL import Image

import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

CHANNEL = 3
DEPTH_IMAGE_WIDTH = 160
DEPTH_IMAGE_HEIGHT = 128
RGB_IMAGE_WIDTH = 304
RGB_IMAGE_HEIGHT = 228
MAX_STEP = 200
MAX_EPOCH = 150
BATCH = 1

In [3]:
def output_predict(predict, kinect, rgb, epoch, step):
	max_val = 10.
	kinect[kinect>max_val] = max_val
	if np.max(kinect) != 0:
		kinect_save = (kinect/max_val)*255.0
		# print('kinect_max', np.amax(kinect))
	else:
		kinect_save = kinect*255.0
	kinect_save=np.uint8(kinect_save)
	name = "data/%04d" % epoch + "_%04d_kinect.png" % step
	cv2.imwrite(name,kinect_save)

	predict[predict>max_val] = max_val
	if np.max(predict) != 0:
		predict_save = (predict/max_val)*255.0
		# print('predict_max', np.amax(predict))
	else:
		predict_save = predict*255.0
	predict_save=np.uint8(predict_save)
	name = "data/%04d" % epoch + "_%04d_predicted.png" % step
	cv2.imwrite(name,predict_save)

	name = "data/%04d" % epoch + "_%04d_rgb.png" % step
	cv2.imwrite(name,rgb)


In [4]:
def SetDiff(first, second):
	second = set(second)
	return [item for item in first if item not in second]

In [5]:
def normalize_rgb(rgb_images, value):
	rgb_images = np.asarray(rgb_images).astype(float)
	rgb_images /= 255.
	for x in range(3):
		rgb_images[:, :, :, x] -= value[x]
	return rgb_images

def consecutive_sample(data, start, end):
	# return a list
	part = []
	for x in range(start, end):
		part.append(data[x])
	return part